In [1]:
# !pip install pandas
# !pip install numpy
# !pip install matplotlib
# !pip install seaborn
# !pip install xarray
# !pip install xclim[extras]
# !pip install openpyxl
# !pip install dash
# !pip install dash-bootstrap-components

In [2]:
# Install necessary libraries (uncomment and run these lines if needed)
# !pip install pandas
# !pip install numpy
# !pip install matplotlib
# !pip install seaborn
# !pip install xarray
# !pip install xclim[extras]
# !pip install openpyxl

# Import required libraries
import pandas as pd  # For handling tabular data
import xarray as xr  # For working with multi-dimensional arrays
import xsdba as sdba  # For statistical bias adjustment (use xsdba directly)
from xclim import set_options  # For configuring xclim options



In [13]:
# -------------------------------
# STEP 1: Load data from Excel
# -------------------------------
# Load observed and model data from Excel files.
# Observed data contains station data (e.g., Edmonton Blatchford).
# Model data contains climate model outputs.
# Both datasets must have a "date" column for time-based operations.

# Load observed data from Excel
obs_df = pd.read_excel("./Data/Edmonton_Blatchford_Joined.xlsx")

# Load model data from Excel
model_df = pd.read_excel("./Data/Edmonton_Tmin_Cannon_16_Compiled .xlsx")

# Convert 'date' columns to datetime format for easier time-based operations
obs_df["date"] = pd.to_datetime(obs_df["DATE"])  # Observed data
model_df["date"] = pd.to_datetime(model_df["ALL DATES"])  # Model data

# Replace anomalous values (-9999.9) with NaN
# These values are placeholders for missing or invalid data.
model_df = model_df.replace(-9999.9, pd.NA)
obs_df = obs_df.replace(-9999.9, pd.NA)

# Handle missing values
# Fill missing values with the column mean to ensure no gaps in the data.
model_df = model_df.fillna(model_df.mean()).infer_objects(copy=False)
obs_df = obs_df.fillna(obs_df.mean()).infer_objects(copy=False)

# Ensure all model columns are numeric
# Convert all columns to numeric, coercing invalid values to NaN.
# model_df = model_df.apply(pd.to_numeric, errors="coerce")

# Handle missing values again (if needed)
model_df = model_df.fillna(model_df.mean()).infer_objects(copy=False)


C:\Users\lucas\AppData\Local\Temp\ipykernel_7776\4048132490.py:26: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



In [14]:
model_df

,ALL DATES,tasmin_KACE-1-0-G_historical_ssp245_(degC),tasmin_GFDL-ESM4_historical_ssp245_(degC),tasmin_NorESM2-LM_historical_ssp245_(degC),tasmin_INM-CM5-0_historical_ssp245_(degC),tasmin_KIOST-ESM_historical_ssp245_(degC),tasmin_CMCC-ESM2_historical_ssp245_(degC),tasmin_FGOALS-g3_historical_ssp245_(degC),tasmin_BCC-CSM2-MR_historical_ssp245_(degC),tasmin_CNRM-ESM2-1_historical_ssp245_(degC),tasmin_MIROC-ES2L_historical_ssp245_(degC),tasmin_MIROC6_historical_ssp245_(degC),tasmin_MPI-ESM1-2-LR_historical_ssp245_(degC),tasmin_MRI-ESM2-0_historical_ssp245_(degC),tasmin_ACCESS-ESM1-5_historical_ssp245_(degC),tasmin_MPI-ESM1-2-HR_historical_ssp245_(degC),tasmin_ACCESS-CM2_historical_ssp245_(degC),date
0,1950-01-01,-9.181352,-30.180815,-13.112076,-18.583960,-20.454716,-39.134808,-12.663461,-40.544743,-16.960400,-18.751810,-12.123291,-18.416111,-19.832148,-22.426182,-17.308306,-26.524754,1950-01-01
1,1950-01-02,-9.440756,-33.879600,-36.311890,-16.069273,-16.816965,-19.978634,-14.332797,-29.240860,-25.944910,-17.949184,-18.739601,-25.474934,-13.490500,-8.134583,-19.349962,-23.173874,1950-01-02
2,1950-01-03,-9.828335,-33.574420,-40.154110,-10.536353,-29.964136,-21.965359,-19.053938,-11.598381,-32.323180,-12.074462,-12.135498,-29.167616,-11.674676,-10.258639,-15.828181,-17.439533,1950-01-03
3,1950-01-04,-8.775462,-36.208126,-37.618060,-6.990157,-24.943920,-26.408785,-24.174864,-15.608451,-31.612110,-10.130463,-14.402988,-11.454946,-17.686730,-9.462118,-12.626839,-24.458683,1950-01-04
4,1950-01-05,-18.693825,-21.641867,-38.063625,-13.469136,-25.969326,-24.065000,-26.802467,-18.532080,-22.877848,-11.345081,-17.665367,-9.977873,-32.683292,-11.808955,-9.102005,-31.764702,1950-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55147,2100-12-27,-5.369649,-4.335087,-7.872128,-30.916300,-14.436559,-22.102690,-11.842526,-2.012665,-3.624017,-17.714195,-10.341038,-12.361332,-24.306093,-8.717478,-4.942397,-11.027694,2100-12-27
55148,2100-12-28,-3.327992,-7.188524,-13.270769,-31.071941,-16.453800,-24.330507,-7.960630,-3.254749,-6.877240,-18.941021,-10.621803,-12.227053,-16.557562,-7.624932,-6.907758,-9.932096,2100-12-28
55149,2100-12-29,-6.626992,-9.733728,-8.964674,-30.257110,-18.852518,-19.716179,-5.882352,-3.904783,-8.577095,-18.397800,-8.851757,-12.300296,-13.768213,-17.326616,-10.313571,-9.068436,2100-12-29
55150,2100-12-30,-8.900586,-5.766384,-18.687721,-31.773857,-14.604407,-17.409016,-10.325779,-4.539558,-5.503928,-17.689781,-9.584190,-9.324787,-11.418324,-13.878078,-9.785609,-12.407109,2100-12-30


In [ ]:
# -------------------------------
# STEP 2: Convert all model columns to a single xarray.DataArray --- https://docs.xarray.dev/en/stable/
# -------------------------------
# Combine all model columns (except the 'date' column) into a single DataArray.
# This allows us to perform bias adjustment on all columns simultaneously.

# Extract all model columns except 'date'
model_columns = [col for col in model_df.columns if col != "ALL DATES"]

# Create a DataArray for all model columns
model_tmin_all = xr.DataArray(
    model_df[model_columns].values.astype(float),  # Ensure numeric data
    dims=("time", "model"),  # Dimensions: time and model
    coords={
        "time": model_df["date"],  # Time coordinates
        "model": model_columns,  # Model names as coordinates
    },
    attrs={"units": "degC"},  # Metadata (units)
)

# Create a DataArray for observed Tmin (single column)
obs_tmin = xr.DataArray(
    obs_df["Tmin"].values,  # Data values
    dims="time",  # Dimension name
    coords={"time": obs_df["date"]},  # Time coordinates
    attrs={"units": "degC"},  # Metadata (units)
)

In [15]:
# -------------------------------
# STEP 3: Define training periods
# -------------------------------
# Training periods are time windows used to train the bias adjustment models.
# These periods are defined as slices of time.

train_slice_30yr = slice("1995-01-01", "2024-12-31")  # 30-year window
train_slice_50yr = slice("1975-01-01", "2024-12-31")  # 50-year window
train_slice_70yr = slice("1955-01-01", "2024-12-31")  # 70-year window

In [16]:
# =====================================================
# STEP 4: Train quantile mapping models --- https://xsdba.readthedocs.io/en/stable/index.html
# =====================================================
# Quantile mapping is a statistical method to adjust biases in model data.
# It aligns the distribution of model data with observed data.
# xclim provides a built-in method called "EmpiricalQuantileMapping" (EQM).

# Check the time range of the data
# This ensures that the observed and model data overlap in time.
# print("Observed time range:", obs_tmin["time"].min().values, "to", obs_tmin["time"].max().values)
# print("Model time range:", model_tmin_all["time"].min().values, "to", model_tmin_all["time"].max().values)

# Ensure time coordinates are in datetime64 format
obs_tmin["time"] = pd.to_datetime(obs_tmin["time"].values)
model_tmin_all["time"] = pd.to_datetime(model_tmin_all["time"].values)

# Select data for the training period (30-year window)
obs_30 = obs_tmin.sel(time=train_slice_30yr)
mod_30 = model_tmin_all.sel(time=train_slice_30yr)

# Check if the selected data is empty
if obs_30.size == 0 or mod_30.size == 0:
    raise ValueError("The selected training period does not overlap with the data.")

# Align time steps
# Align observed and model data along the time dimension.
obs_30, mod_30 = xr.align(obs_30, mod_30, join="inner")

# Train the EQM model for the 30-year window
QM_30 = sdba.EmpiricalQuantileMapping.train(
    ref=obs_30,  # Observed data (reference)
    hist=mod_30,  # Model data (historical)
)

# Repeat the process for the 50-year window
obs_50 = obs_tmin.sel(time=train_slice_50yr)
mod_50 = model_tmin_all.sel(time=train_slice_50yr)
obs_50, mod_50 = xr.align(obs_50, mod_50, join="inner")
QM_50 = sdba.EmpiricalQuantileMapping.train(ref=obs_50, hist=mod_50)

# Repeat the process for the 70-year window
obs_70 = obs_tmin.sel(time=train_slice_70yr)
mod_70 = model_tmin_all.sel(time=train_slice_70yr)
obs_70, mod_70 = xr.align(obs_70, mod_70, join="inner")
QM_70 = sdba.EmpiricalQuantileMapping.train(ref=obs_70, hist=mod_70)

In [19]:
# =====================================================
# STEP 5: Apply corrections
# =====================================================
# Once trained, the EQM models can be applied to adjust all model data.
# This step corrects the model values for biases.

# Apply the 30-year EQM model to adjust Tmin
tmin_corrected_30 = QM_30.adjust(model_tmin_all)

# Apply the 50-year EQM model to adjust Tmin
tmin_corrected_50 = QM_50.adjust(model_tmin_all)

# Apply the 70-year EQM model to adjust Tmin
tmin_corrected_70 = QM_70.adjust(model_tmin_all)

# ================================
# Export corrected data to Excel
# ================================
# Convert DataArrays to DataFrames for Excel export
# tmin_corrected_30_df = pd.DataFrame(
#     tmin_corrected_30.values,
#     columns=tmin_corrected_30.model.values,
#     index=pd.to_datetime(tmin_corrected_30.time.values)
# )
# tmin_corrected_30_df.index.name = "date"

# tmin_corrected_50_df = pd.DataFrame(
#     tmin_corrected_50.values,
#     columns=tmin_corrected_50.model.values,
#     index=pd.to_datetime(tmin_corrected_50.time.values)
# )
# tmin_corrected_50_df.index.name = "date"

# tmin_corrected_70_df = pd.DataFrame(
#     tmin_corrected_70.values,
#     columns=tmin_corrected_70.model.values,
#     index=pd.to_datetime(tmin_corrected_70.time.values)
# )
# tmin_corrected_70_df.index.name = "date"

# # Write to Excel files
# tmin_corrected_30_df.to_excel("./Data/tmin_corrected_30yr.xlsx")
# tmin_corrected_50_df.to_excel("./Data/tmin_corrected_50yr.xlsx")
# tmin_corrected_70_df.to_excel("./Data/tmin_corrected_70yr.xlsx")


In [20]:
# =====================================================
# STEP 6: Exceedance probability
# =====================================================
# Example: What is the probability that Tmin exceeds a threshold (e.g., 15°C)?
# We calculate this for raw model data and compare it with bias-adjusted results.

threshold = 15  # Threshold temperature in degrees Celsius

# Calculate the probability of exceeding the threshold for raw model data
prob_raw_all = (model_tmin_all > threshold).mean(dim="time")

# Calculate the probability for bias-adjusted data for each window
prob_corrected_30 = (tmin_corrected_30 > threshold).mean(dim="time")
prob_corrected_50 = (tmin_corrected_50 > threshold).mean(dim="time")
prob_corrected_70 = (tmin_corrected_70 > threshold).mean(dim="time")


In [22]:

# # =====================================================
# # STEP 7: Visualize results with Dash
# # =====================================================
# # We use Dash to create an interactive web app for visualizing the results.

# import dash  # For creating web apps
# from dash import dcc, html, dash_table  # For app components
# import plotly.graph_objs as go  # For creating plots

# # -------------------------------
# # STEP 1: Slice data for plotting
# # -------------------------------
# # Define the time range for plotting
# start_plot = "2025-01-01"
# end_plot = "2030-12-31"

# # Slice the raw and corrected data for the plotting range
# model_subset = model_tmin_all.sel(time=slice(start_plot, end_plot))
# tmin_30_subset = tmin_corrected_30.sel(time=slice(start_plot, end_plot))
# tmin_50_subset = tmin_corrected_50.sel(time=slice(start_plot, end_plot))
# tmin_70_subset = tmin_corrected_70.sel(time=slice(start_plot, end_plot))
# obs_subset = obs_tmin.sel(time=slice(start_plot, end_plot))

# # -------------------------------
# # STEP 2: Create Dash app
# # -------------------------------
# # Initialize the Dash app
# app = dash.Dash(__name__)

# # Define the layout of the app
# app.layout = html.Div([
#     html.H2("Tmin Comparison: Raw vs Bias-Adjusted Models (2025-2030)"),  # Title
#     dcc.Dropdown(  # Dropdown menu for selecting models
#         id="model-dropdown",
#         options=[{"label": model, "value": model} for model in model_columns],
#         value=model_columns[0],  # Default value (first model)
#         clearable=False,
#     ),
#     html.Div([
#         html.Label("Exceedance Threshold (°C):"),
#         dcc.Input(
#             id="threshold-input",
#             type="number",
#             value=15,  # Default threshold
#             step=0.1,
#         ),
#     ], style={"margin-bottom": "20px"}),
#     dcc.Graph(id="tmin-graph"),  # Graph component
#     html.H3("Exceedance Probabilities"),  # Table title
#     dash_table.DataTable(  # Table to display probabilities
#         id="probability-table",
#         columns=[
#             {"name": "Model", "id": "model"},
#             {"name": "Raw", "id": "raw"},
#             {"name": "Bias-Adjusted (30yr)", "id": "30yr"},
#             {"name": "Bias-Adjusted (50yr)", "id": "50yr"},
#             {"name": "Bias-Adjusted (70yr)", "id": "70yr"},
#         ],
#         style_table={"overflowX": "auto"},
#         style_cell={"textAlign": "center"},
#     )
# ])

# # -------------------------------
# # STEP 3: Define callbacks
# # -------------------------------
# @app.callback(
#     [dash.dependencies.Output("tmin-graph", "figure"),
#      dash.dependencies.Output("probability-table", "data")],
#     [dash.dependencies.Input("model-dropdown", "value"),
#      dash.dependencies.Input("threshold-input", "value")]
# )
# def update_graph_and_table(selected_model, threshold):
#     # Select the data for the chosen model
#     raw_data = model_subset.sel(model=selected_model)
#     data_30yr = tmin_30_subset.sel(model=selected_model)
#     data_50yr = tmin_50_subset.sel(model=selected_model)
#     data_70yr = tmin_70_subset.sel(model=selected_model)

#     # Get exceedance probabilities for the selected model
#     prob_raw = (raw_data > threshold).mean().item()
#     prob_30 = (data_30yr > threshold).mean().item()
#     prob_50 = (data_50yr > threshold).mean().item()
#     prob_70 = (data_70yr > threshold).mean().item()

#     # Create the figure
#     figure = {
#         "data": [
#             # Plot observed Tmin
#             go.Scatter(
#                 x=obs_subset.time.values,
#                 y=obs_subset.values,
#                 mode="lines",
#                 name="Observed Tmin",
#                 line=dict(color="red", width=2, dash="dash")
#             ),
#             # Plot raw model Tmin
#             go.Scatter(
#                 x=raw_data.time.values,
#                 y=raw_data.values,
#                 mode="lines",
#                 name=f"Raw Model Tmin ({selected_model})",
#                 line=dict(color="gray", width=2)
#             ),
#             # Plot 30-year bias-adjusted Tmin
#             go.Scatter(
#                 x=data_30yr.time.values,
#                 y=data_30yr.values,
#                 mode="lines",
#                 name="Bias-Adjusted (30yr)",
#                 line=dict(color="blue", width=2)
#             ),
#             # Plot 50-year bias-adjusted Tmin
#             go.Scatter(
#                 x=data_50yr.time.values,
#                 y=data_50yr.values,
#                 mode="lines",
#                 name="Bias-Adjusted (50yr)",
#                 line=dict(color="purple", width=2)
#             ),
#             # Plot 70-year bias-adjusted Tmin
#             go.Scatter(
#                 x=data_70yr.time.values,
#                 y=data_70yr.values,
#                 mode="lines",
#                 name="Bias-Adjusted (70yr)",
#                 line=dict(color="cyan", width=2)
#             )
#         ],
#         "layout": go.Layout(
#             xaxis={"title": "Time"},  # X-axis label
#             yaxis={"title": "Tmin (°C)"},  # Y-axis label
#             template="plotly_white",  # Plot style
#             hovermode="x unified"  # Unified hover mode
#         )
#     }

#     # Create the table data
#     table_data = [{
#         "model": selected_model,
#         "raw": f"{prob_raw:.3f}",
#         "30yr": f"{prob_30:.3f}",
#         "50yr": f"{prob_50:.3f}",
#         "70yr": f"{prob_70:.3f}",
#     }]

#     return figure, table_data

# # Run the app
# if __name__ == "__main__":
#     app.run(debug=True)  # Enable debug mode for easier troubleshooting